# Introduction to LangGraph & State Management

In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import os

print("All libraries imported!")

All libraries imported!


In [3]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API Key not found!")
print("API Key loaded successfully")

API Key loaded successfully


In [4]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.7,
    api_key=api_key
)

print(f"LLM initialized {llm.model_name}")

LLM initialized gpt-4o-mini


## State

State is the data that flows through the graph<br>
MessagesState: stores conversation history<br>

Nodes processes the state and return updates, they are defined the way functions are defined

### The Assistant Node

In [ ]:
# A node that send messages to the LLM and gets a response
sys_msg = SystemMessage(
    content= "You are a friendly assistant that answers user question. Be helpful and concise."
)

def assitant(state: MessagesState) -> dict: #state expect an input in the form of MessagesState, and gives a dictionary as the output
    """
    The assistant node - processes messages and generates response.
    """
    